# Counting the route step coordinates

This script is to count all OSRM route step coordinates from the training data set. The training data set has 1.4 million trips as each trip has multiple step coordinates, the output CSV will count more than 2 million coordinates. For instance, the output CSV file shows the maximum counting number is 43763 and its coordinate is -73.973249, 40.739237, which means there are 43763 trip routes from the training data that pass this coordinate. 


In [1]:
import findspark
findspark.init('c:/spark')
from pyspark import SparkConf, SparkContext
import collections
import re
from pyspark.sql import SparkSession
from pyspark.sql import Row
import pandas as pd
import collections

In [2]:
conf = SparkConf().setMaster("local").setAppName("Coordinates")
sc = SparkContext(conf = conf)

In [5]:
lines = sc.textFile("train_full_parsed_clean3.1.csv")

header = lines.first()

header1= sc.parallelize([header])
linesnoheader = lines.subtract(header1)

In [6]:
def loaddirection(line):
    
    coordinates=re.findall('(\[\-\d+\.\d+\,\s\d+\.\d+\])',line)
    return coordinates

In [7]:
direction = lines.flatMap(loaddirection)
direction

PythonRDD[17] at RDD at PythonRDD.scala:48

In [8]:
wordCounts = direction.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)
wordCountsSorted = wordCounts.map(lambda x: (x[1], x[0])).sortByKey(False)
results = wordCountsSorted.collect()

In [9]:
df=pd.DataFrame(results,columns=['count','coordinates'])

In [10]:
df['longitude']=df['coordinates'].apply(lambda x: re.findall('(-?\d+\.\d+)',x)[0])
df['latitude']=df['coordinates'].apply(lambda x: re.findall('(-?\d+\.\d+)',x)[1])

In [11]:
df.head(10)

,count,coordinates,longitude,latitude
0,43763,"[-73.973249, 40.739237]",-73.973249,40.739237
1,37907,"[-73.980185, 40.71098]",-73.980185,40.71098
2,35343,"[-73.974467, 40.722333]",-73.974467,40.722333
3,29042,"[-73.957928, 40.761162]",-73.957928,40.761162
4,29013,"[-73.97191, 40.743309]",-73.97191,40.743309
5,28462,"[-73.972169, 40.72622]",-73.972169,40.72622
6,27783,"[-73.972049, 40.726193]",-73.972049,40.726193
7,26944,"[-73.973878, 40.731708]",-73.973878,40.731708
8,26435,"[-73.975683, 40.728454]",-73.975683,40.728454
9,26309,"[-73.975064, 40.735925]",-73.975064,40.735925


In [16]:
df[['count','longitude','latitude']].to_csv("newcounter.csv",index=False)